# Infer-7-Classification : Classification Bayesienne

**Serie** : Programmation Probabiliste avec Infer.NET (7/12)  
**Duree estimee** : 50 minutes  
**Prerequis** : Infer-6-TrueSkill

---

## Objectifs

- Implementer la regression logistique bayesienne
- Comprendre le Bayes Point Machine (BPM)
- Appliquer l'inference bayesienne aux tests cliniques (A/B testing)
- Gerer l'incertitude dans les predictions

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-6-TrueSkill](Infer-6-TrueSkill.ipynb) | [Infer-8-Model-Selection](Infer-8-Model-Selection.ipynb) |

---

## 1. Configuration

In [ ]:
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET pret !");

## 2. Classification Probabiliste

### Difference avec la classification classique

| Approche | Sortie | Incertitude |
|----------|--------|-------------|
| **Classique** | Classe predite | Non |
| **Probabiliste** | P(classe) | Oui |
| **Bayesienne** | Distribution sur P(classe) | Oui + incertitude sur le modele |

### Avantages bayesiens

- Quantification de l'incertitude
- Regularisation naturelle (priors)
- Mise a jour incrementale
- Pas de surapprentissage si bon prior

## 3. Regression Logistique Bayesienne (1 feature)

In [ ]:
// Donnees : classification binaire avec 1 feature
double[] features = { 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0 };
bool[] labels = { false, false, false, true, false, true, true, true };
int n = features.Length;

// Modele : y = sigmoid(w * x - b)
// Equivalent probit : y = I(w * x - b + noise > 0)

Variable<double> poids = Variable.GaussianFromMeanAndVariance(0, 10).Named("poids");
Variable<double> seuil = Variable.GaussianFromMeanAndVariance(0, 10).Named("seuil");
Variable<double> bruitPrecision = Variable.GammaFromShapeAndScale(2, 0.5).Named("bruit");

Range dataRange = new Range(n);
VariableArray<double> xObs = Variable.Array<double>(dataRange).Named("x");
VariableArray<bool> yObs = Variable.Array<bool>(dataRange).Named("y");

using (Variable.ForEach(dataRange))
{
    Variable<double> score = poids * xObs[dataRange] - seuil;
    Variable<double> scoreBruite = Variable.GaussianFromMeanAndPrecision(score, bruitPrecision);
    yObs[dataRange] = (scoreBruite > 0);
}

xObs.ObservedValue = features;
yObs.ObservedValue = labels;

InferenceEngine moteur = new InferenceEngine(new ExpectationPropagation());
moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Gaussian poidsPost = moteur.Infer<Gaussian>(poids);
Gaussian seuilPost = moteur.Infer<Gaussian>(seuil);

Console.WriteLine("=== Regression Logistique Bayesienne ===");
Console.WriteLine($"\nPoids : {poidsPost}");
Console.WriteLine($"Seuil : {seuilPost}");
Console.WriteLine($"\nInterpretation : classe 1 si feature > {seuilPost.GetMean() / poidsPost.GetMean():F2}");

## 4. Prediction avec Incertitude

In [ ]:
// Prediction pour de nouvelles valeurs
double[] nouveauxX = { 2.5, 4.5, 6.5, 9.0 };

Console.WriteLine("=== Predictions avec incertitude ===");
Console.WriteLine();

foreach (double x in nouveauxX)
{
    // Modele de prediction
    Variable<Gaussian> poidsPrior = Variable.Observed(poidsPost);
    Variable<Gaussian> seuilPrior = Variable.Observed(seuilPost);
    
    Variable<double> poidsPred = Variable.Random<double, Gaussian>(poidsPrior);
    Variable<double> seuilPred = Variable.Random<double, Gaussian>(seuilPrior);
    Variable<double> bruitPred = Variable.GammaFromShapeAndScale(2, 0.5);
    
    Variable<double> scorePred = poidsPred * x - seuilPred;
    Variable<double> scoreBruitePred = Variable.GaussianFromMeanAndPrecision(scorePred, bruitPred);
    Variable<bool> predLabel = (scoreBruitePred > 0);
    
    InferenceEngine moteurPred = new InferenceEngine(new ExpectationPropagation());
    moteurPred.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    
    Bernoulli prediction = moteurPred.Infer<Bernoulli>(predLabel);
    Console.WriteLine($"x = {x:F1} : P(classe=1) = {prediction.GetProbTrue():F3}");
}

## 5. Classification Multi-Features

In [ ]:
// Donnees avec 2 features
double[,] featuresMulti = {
    { 1.0, 2.0 },
    { 2.0, 1.5 },
    { 1.5, 3.0 },
    { 3.0, 3.5 },
    { 4.0, 2.0 },
    { 3.5, 4.0 },
    { 5.0, 3.0 },
    { 4.5, 5.0 }
};
bool[] labelsMulti = { false, false, false, true, false, true, true, true };

int nSamples = labelsMulti.Length;
int nFeatures = 2;

Range sampleRange = new Range(nSamples).Named("sample");
Range featureRange = new Range(nFeatures).Named("feature");

// Poids pour chaque feature
VariableArray<double> poidsMulti = Variable.Array<double>(featureRange).Named("poids");
poidsMulti[featureRange] = Variable.GaussianFromMeanAndVariance(0, 10).ForEach(featureRange);

Variable<double> seuilMulti = Variable.GaussianFromMeanAndVariance(0, 10).Named("seuil");

VariableArray2D<double> xMulti = Variable.Array<double>(sampleRange, featureRange).Named("x");
VariableArray<bool> yMulti = Variable.Array<bool>(sampleRange).Named("y");

using (Variable.ForEach(sampleRange))
{
    // Score = somme(poids[f] * x[f]) - seuil
    Variable<double> scoreMulti = Variable.Sum(
        Variable.Array<double>(featureRange).Named("produit"));
    
    // Alternative plus simple : calculer explicitement
    Variable<double> score0 = poidsMulti[0] * xMulti[sampleRange, 0];
    Variable<double> score1 = poidsMulti[1] * xMulti[sampleRange, 1];
    Variable<double> scoreTot = score0 + score1 - seuilMulti;
    Variable<double> scoreNoise = Variable.GaussianFromMeanAndVariance(scoreTot, 1);
    yMulti[sampleRange] = (scoreNoise > 0);
}

xMulti.ObservedValue = featuresMulti;
yMulti.ObservedValue = labelsMulti;

InferenceEngine moteurMulti = new InferenceEngine(new ExpectationPropagation());
moteurMulti.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Gaussian[] poidsPostMulti = moteurMulti.Infer<Gaussian[]>(poidsMulti);
Gaussian seuilPostMulti = moteurMulti.Infer<Gaussian>(seuilMulti);

Console.WriteLine("=== Classification Multi-Features ===");
for (int f = 0; f < nFeatures; f++)
{
    Console.WriteLine($"Poids feature {f+1} : {poidsPostMulti[f]}");
}
Console.WriteLine($"Seuil : {seuilPostMulti}");

## 6. Bayes Point Machine (BPM)

### Principe

Le BPM est une methode de classification bayesienne qui :
- Marginalise sur tous les hyperplans separateurs possibles
- Donne des probabilites calibrees
- Utilise EP pour l'inference

### Formulation

$$P(y=1|x) = \int P(y=1|x,w) P(w|D) dw$$

In [ ]:
// Bayes Point Machine simplifie

public class SimpleBPM
{
    private int nFeatures;
    private Gaussian[] poidsPosteriors;
    private Gaussian seuilPosterior;
    private InferenceEngine moteur;
    
    public SimpleBPM(int nFeatures)
    {
        this.nFeatures = nFeatures;
        this.moteur = new InferenceEngine(new ExpectationPropagation());
        this.moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    }
    
    public void Entrainer(double[,] X, bool[] y)
    {
        int n = y.Length;
        Range sampleRange = new Range(n);
        Range featureRange = new Range(nFeatures);
        
        VariableArray<double> poids = Variable.Array<double>(featureRange);
        poids[featureRange] = Variable.GaussianFromMeanAndVariance(0, 1).ForEach(featureRange);
        
        Variable<double> seuil = Variable.GaussianFromMeanAndVariance(0, 1);
        
        VariableArray2D<double> xVar = Variable.Array<double>(sampleRange, featureRange);
        VariableArray<bool> yVar = Variable.Array<bool>(sampleRange);
        
        using (Variable.ForEach(sampleRange))
        {
            Variable<double> score = Variable.Constant(0.0);
            for (int f = 0; f < nFeatures; f++)
            {
                score = score + poids[f] * xVar[sampleRange, f];
            }
            score = score - seuil;
            Variable<double> scoreNoise = Variable.GaussianFromMeanAndVariance(score, 1);
            yVar[sampleRange] = (scoreNoise > 0);
        }
        
        xVar.ObservedValue = X;
        yVar.ObservedValue = y;
        
        poidsPosteriors = moteur.Infer<Gaussian[]>(poids);
        seuilPosterior = moteur.Infer<Gaussian>(seuil);
    }
    
    public double Predire(double[] x)
    {
        // Score moyen
        double scoreMoyen = 0;
        double scoreVariance = 0;
        
        for (int f = 0; f < nFeatures; f++)
        {
            scoreMoyen += poidsPosteriors[f].GetMean() * x[f];
            scoreVariance += poidsPosteriors[f].GetVariance() * x[f] * x[f];
        }
        scoreMoyen -= seuilPosterior.GetMean();
        scoreVariance += seuilPosterior.GetVariance() + 1;  // +1 pour le bruit
        
        // Probit : P(score + noise > 0)
        return Gaussian.CDF(scoreMoyen / Math.Sqrt(scoreVariance));
    }
}

Console.WriteLine("Classe SimpleBPM definie.");

In [ ]:
// Utilisation du BPM
var bpm = new SimpleBPM(2);
bpm.Entrainer(featuresMulti, labelsMulti);

Console.WriteLine("=== Predictions BPM ===");
double[][] testPoints = {
    new[] { 1.0, 1.0 },
    new[] { 3.0, 3.0 },
    new[] { 5.0, 4.0 },
    new[] { 2.0, 4.0 }
};

foreach (var point in testPoints)
{
    double prob = bpm.Predire(point);
    Console.WriteLine($"({point[0]}, {point[1]}) : P(classe=1) = {prob:F3}");
}

## 7. Test Clinique Bayesien (A/B Testing)

### Contexte

Comparer l'efficacite d'un nouveau traitement vs placebo.

### Approche bayesienne

- Prior sur l'efficacite de chaque traitement
- Mise a jour avec les observations
- Probabilite que le traitement soit meilleur

In [ ]:
// Test clinique A/B bayesien

// Donnees observees
int nPlacebo = 100;
int guerisPlacebo = 30;  // 30% guerison

int nTraitement = 100;
int guerisTraitement = 45;  // 45% guerison

// Modele : taux de guerison pour chaque groupe
Variable<double> tauxPlacebo = Variable.Beta(1, 1).Named("tauxPlacebo");  // Prior uniforme
Variable<double> tauxTraitement = Variable.Beta(1, 1).Named("tauxTraitement");

// Observations (distribution binomiale)
Variable<int> obsPlacebo = Variable.Binomial(nPlacebo, tauxPlacebo);
Variable<int> obsTraitement = Variable.Binomial(nTraitement, tauxTraitement);

obsPlacebo.ObservedValue = guerisPlacebo;
obsTraitement.ObservedValue = guerisTraitement;

// Traitement est meilleur ?
Variable<bool> traitementMeilleur = (tauxTraitement > tauxPlacebo);

InferenceEngine moteurClinique = new InferenceEngine();
moteurClinique.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Beta tauxPlaceboPost = moteurClinique.Infer<Beta>(tauxPlacebo);
Beta tauxTraitementPost = moteurClinique.Infer<Beta>(tauxTraitement);
Bernoulli traitementMeilleurPost = moteurClinique.Infer<Bernoulli>(traitementMeilleur);

Console.WriteLine("=== Test Clinique Bayesien ===");
Console.WriteLine($"\nPlacebo : {guerisPlacebo}/{nPlacebo} guerisons");
Console.WriteLine($"Traitement : {guerisTraitement}/{nTraitement} guerisons");

Console.WriteLine($"\nTaux placebo : {tauxPlaceboPost}");
Console.WriteLine($"  Moyenne : {tauxPlaceboPost.GetMean():F3}");
Console.WriteLine($"  IC 95% : [{tauxPlaceboPost.GetMean() - 2*Math.Sqrt(tauxPlaceboPost.GetVariance()):F3}, {tauxPlaceboPost.GetMean() + 2*Math.Sqrt(tauxPlaceboPost.GetVariance()):F3}]");

Console.WriteLine($"\nTaux traitement : {tauxTraitementPost}");
Console.WriteLine($"  Moyenne : {tauxTraitementPost.GetMean():F3}");

Console.WriteLine($"\nP(traitement meilleur) = {traitementMeilleurPost.GetProbTrue():F3}");

### Analyse

- Le traitement semble meilleur (45% vs 30%)
- La probabilite bayesienne quantifie notre confiance
- Pas de p-value arbitraire !

## 8. Effet de la Taille d'Echantillon

In [ ]:
// Impact de la taille d'echantillon

Console.WriteLine("=== Impact de la taille d'echantillon ===");
Console.WriteLine("\nMeme ratio (30% vs 45%), differentes tailles :\n");

int[] tailles = { 10, 50, 100, 500, 1000 };

foreach (int n in tailles)
{
    int gP = (int)(n * 0.30);
    int gT = (int)(n * 0.45);
    
    Variable<double> tP = Variable.Beta(1, 1);
    Variable<double> tT = Variable.Beta(1, 1);
    
    Variable.ConstrainEqual(Variable.Binomial(n, tP), gP);
    Variable.ConstrainEqual(Variable.Binomial(n, tT), gT);
    
    Variable<bool> meilleur = (tT > tP);
    
    InferenceEngine m = new InferenceEngine();
    m.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    
    double prob = m.Infer<Bernoulli>(meilleur).GetProbTrue();
    Console.WriteLine($"n = {n,4} : P(traitement meilleur) = {prob:F4}");
}

Console.WriteLine("\n=> Plus de donnees = plus de certitude");

## 9. Exercice : Classification Spam

### Enonce

Construisez un classificateur bayesien pour detecter les spams bases sur 3 features :
- Nombre de mots en majuscules
- Presence du mot "gratuit"
- Longueur du message (en centaines de caracteres)

### Donnees

In [ ]:
// EXERCICE : Classification spam

// Donnees d'entrainement
// [nbMajuscules, presenceGratuit (0/1), longueur]
double[,] spamFeatures = {
    { 5, 0, 2.5 },   // Non spam
    { 3, 0, 3.0 },   // Non spam
    { 15, 1, 1.0 },  // Spam
    { 20, 1, 0.5 },  // Spam
    { 2, 0, 4.0 },   // Non spam
    { 25, 1, 1.5 },  // Spam
    { 8, 0, 2.0 },   // Non spam
    { 18, 1, 0.8 }   // Spam
};
bool[] spamLabels = { false, false, true, true, false, true, false, true };

// Entrainement
var spamClassifier = new SimpleBPM(3);
spamClassifier.Entrainer(spamFeatures, spamLabels);

Console.WriteLine("=== Classificateur Spam ===");

// Test sur nouveaux emails
var testEmails = new (double[], string)[] {
    (new[] { 4.0, 0.0, 3.0 }, "Email normal"),
    (new[] { 22.0, 1.0, 1.0 }, "OFFRE GRATUITE!!!"),
    (new[] { 10.0, 0.0, 2.5 }, "Email avec quelques majuscules"),
    (new[] { 30.0, 1.0, 0.5 }, "CLIQUEZ ICI GRATUIT")
};

Console.WriteLine("\nPredictions :");
foreach (var (features, desc) in testEmails)
{
    double probSpam = spamClassifier.Predire(features);
    string verdict = probSpam > 0.5 ? "SPAM" : "OK";
    Console.WriteLine($"  {desc,-30} : P(spam)={probSpam:F3} -> {verdict}");
}

## 10. Resume

| Concept | Description |
|---------|-------------|
| **Regression logistique bayesienne** | Priors sur poids, posterieurs apres donnees |
| **Bayes Point Machine** | Marginalisation sur hyperplans |
| **Test A/B bayesien** | P(traitement meilleur) directement |
| **Incertitude** | Quantifiee a chaque etape |
| **Calibration** | Probabilites refletent la vraie incertitude |

---

## Prochaine etape

Dans [Infer-8-Model-Selection](Infer-8-Model-Selection.ipynb), nous explorerons :

- La selection et comparaison de modeles
- L'evidence bayesienne (marginal likelihood)
- Le facteur de Bayes
- L'Automatic Relevance Determination (ARD)